In [1]:
import ctypes
import numpy as np
from numba import cuda

In [2]:
# a=np.arange(6).reshape(2,3).astype(np.float32)
a = np.random.rand(2,3).astype(np.float32)
# ad=a.ravel('F')
af=np.asfortranarray(a)

In [3]:
# b=np.arange(12).reshape(3,4).astype(np.float32)
b = np.random.rand(3,4).astype(np.float32)
# bd=b.ravel('F')
bf=np.asfortranarray(b)

In [4]:
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32)
# cd=c.ravel('F')
cf=np.asfortranarray(c)

In [5]:
af.shape,bf.shape,cf.shape

((2, 3), (3, 4), (2, 4))

In [6]:
a.shape,b.shape,c.shape

((2, 3), (3, 4), (2, 4))

In [7]:
sgemm=ctypes.CDLL('libsgemm.so')
al,bet,b0=1,0,np.arange(1,dtype=np.float32)

In [9]:
%%time
da=cuda.to_device(a).T
db=cuda.to_device(b).T
# dc=cuda.to_device(c).T
dc=cuda.device_array_like(c.T)

CPU times: user 333 ms, sys: 50.2 ms, total: 383 ms
Wall time: 414 ms


In [10]:
m=a.shape[0]
n=b.shape[1]
k=b.shape[0]
lda=a.shape[0]
ldb=b.shape[0]
ldc=c.shape[0]
print(m,n,k)
print(lda,ldb,ldc)

2 4 3
2 3 2


In [11]:
da.shape,db.shape,dc.shape

((3, 2), (4, 3), (4, 2))

In [12]:
%%time
sgemm.createHandle()
sgemm.gemm(ord('N'),ord('N'),
           m,n,k,
           ctypes.c_float(al),
           da.device_ctypes_pointer, lda,
           db.device_ctypes_pointer, ldb,
           ctypes.c_float(bet),
           dc.device_ctypes_pointer, ldc,
           ctypes.c_void_p(b0.ctypes.data))
sgemm.destroyHandle()

CPU times: user 165 ms, sys: 79.9 ms, total: 245 ms
Wall time: 244 ms


In [13]:
%%time
d=a.dot(b)

CPU times: user 357 µs, sys: 42 µs, total: 399 µs
Wall time: 272 µs


In [14]:
cd=dc.copy_to_host().reshape(d.shape,order='A')

In [15]:
cd

array([[0.8548593 , 0.2530516 , 0.7698976 , 1.2938049 ],
       [1.0392545 , 0.13117808, 0.8047702 , 0.927003  ]], dtype=float32)

In [16]:
d

array([[0.8548593 , 0.2530516 , 0.7698976 , 1.2938049 ],
       [1.0392545 , 0.13117808, 0.8047702 , 0.927003  ]], dtype=float32)

In [17]:
abs(d-cd).sum(),abs(d-cd).mean(),abs(d-cd).max()

(0.0, 0.0, 0.0)

In [18]:
cd.ravel('K')-d.ravel('F')

array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [19]:
np.allclose(d,cd)

True

In [2]:
gg=np.empty((11900,20000),dtype=np.float32)
gg[:]=np.random.rand(*gg.shape)
gg.size*4/1024/1024

907.89794921875

In [3]:
%%time
dgg=cuda.to_device(gg)#.T.reshape(gg.shape)

CPU times: user 428 ms, sys: 194 ms, total: 622 ms
Wall time: 662 ms


In [4]:
%%time
dgt=cuda.device_array_like(gg)

CPU times: user 0 ns, sys: 2.36 ms, total: 2.36 ms
Wall time: 1.74 ms


In [5]:
sgemm=ctypes.CDLL('libsgemm.so')
al,bet=1,0
m,n=gg.shape

In [6]:
%%time
sgemm.createHandle()
sgemm.transpose(dgg.device_ctypes_pointer,
                dgt.device_ctypes_pointer,
                ctypes.c_int(n),ctypes.c_int(m),
                ctypes.c_float(al),ctypes.c_float(bet))
sgemm.destroyHandle()

CPU times: user 294 ms, sys: 108 ms, total: 402 ms
Wall time: 534 ms


In [7]:
(dgt.copy_to_host().ravel('K')==gg.ravel('F')).all()

True

In [12]:
del dgg
del dgt